In [2]:
import time
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
from scipy import signal
from prettytable import PrettyTable

# Relaxation Method
1) Divide the region of interest into a rectangular grid spanning the region. The region is enclosed by a surface (curve in two dimensions) with specified values of the potential along the curve.

2) Assign to a boundary site the potential of the boundary nearest the site.

3) Assign all interior sites an arbitrary potential (preferably a reasonable guess).

4) Compute new values for the potential V for each interior site. Each new value is obtained by finding the average of the previous values of the potential at the four nearest neighbor sites.

5) Repeat step (4) using the values of V obtained in the previous iteration. This iterative process is continued until the potential at each interior site is computed to the desired accuracy.

# Problem 10.10 Numerical solution of the potential within a rectangular region

In [3]:
# Part (a)
# Part 1 of the relaxation method: create a rectangular grid spanning the region
L = 6
dx = 1
dy = 1
boundary_left = 5
boundary_right = 5
boundary_top = 10
boundary_bottom = 10
threshold = 1e-16

def V(i, j, u):
    return (1/4) * (u[i+1, j] + u[i-1, j] + u[i, j+1] + u[i, j-1])

# Part 3: create our initial guess u_0
u0 = np.zeros([L, L])

# Part 2: assign to a boundary site the potential of the boundary nearest the site
def assign_boundary(u, boundary_left, boundary_right, boundary_top, boundary_bottom):
    u[:, 0] = boundary_left  # left col
    u[:, -1] = boundary_right # right col
    u[0, :] = boundary_top  # top row
    u[-1, :] = boundary_bottom # bottom row
    return u
u0 = assign_boundary(u0, boundary_left, boundary_right, boundary_top, boundary_bottom)

# Part 4: compute new values for the potential V for each interior site
def solve_u(u, do_step, threshold, max_iters=10000):
    start_time = time.time()
    error = 100
    iters = 0
    u_prev = np.zeros(u.shape)
    while error > threshold and iters < max_iters:
        u = do_step(u)
        error = compute_error(u, u_prev)
        u_prev = np.copy(u)
        iters += 1
    return u, iters, error, (time.time() - start_time)
        
# 5) Repeat step (4) using the values of V obtained in the previous iteration. 
# This iterative process is continued until the potential at each interior site 
# is computed to the desired accuracy.        
def compute_error(u, u_last):
    diffs = np.abs(u - u_last)
    return diffs.mean()

In [4]:
# Define different solving methods using explicit method
def do_step_basic(u):
    temp = np.copy(u)
    for i in range(1, L-1):
        for j in range(1, L-1):
            temp[i, j] = V(i, j, u)
    return temp

def do_step_in_place(u):
    rows, cols = u.shape
    for i in range(1, rows-1):
        for j in range(1, cols-1):
            u[i, j] = V(i, j, u)
    return u

def do_step_convolve(u):
    kernel = np.array([[0, 1/4, 0], [1/4, 0, 1/4], [0, 1/4, 0]])
    u_temp = signal.convolve2d(u, kernel, mode='same')
    return assign_boundary(u_temp, boundary_left, boundary_right, boundary_top, boundary_bottom)

In [5]:
"""
Run all 3 methods and compare number of steps to converge, error, and total time for each
"""
# One step at a time
u_solution, iterations_basic, error_basic, time_basic = solve_u(u0, do_step_basic, threshold)

# Change values in place
u_solution, iterations_inplace, error_inplace, time_inplace = solve_u(u0, do_step_in_place, threshold)

# Compute changes via convolution
u_solution, iterations_convolve, error_convolve, time_convolve = solve_u(u0, do_step_convolve, threshold)

# Plot the results in a table
t = PrettyTable()
t.title = 'Comparison of Explicit FDM Methods'
t.field_names = ['Method', 'Iterations to Convergence', 'Time to Convergence (s)', 'Error']
t.add_row(['One step at a time', iterations_basic, time_basic, error_basic])
t.add_row(['Changes in place', iterations_inplace, time_inplace, error_inplace])
t.add_row(['Convolution', iterations_convolve, time_convolve, error_convolve])
print(t)

+---------------------------------------------------------------------------------------------------+
|                                 Comparison of Explicit FDM Methods                                |
+--------------------+---------------------------+-------------------------+------------------------+
|       Method       | Iterations to Convergence | Time to Convergence (s) |         Error          |
+--------------------+---------------------------+-------------------------+------------------------+
| One step at a time |            171            |   0.014852046966552734  | 9.868649107779169e-17  |
|  Changes in place  |             87            |  0.0024461746215820312  | 4.9343245538895844e-17 |
|    Convolution     |             4             |  0.0002200603485107422  | 4.9343245538895844e-17 |
+--------------------+---------------------------+-------------------------+------------------------+


# Problem 10.11 Gauss-Seidel relaxation